# 1. Connect to IB 

In [ ]:

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

# 2. Market Segment Analysis

In [ ]:
import market_analyzer as ma

# Initialize analyzer
analyzer = ma.IBMarketAnalyzer(ib)

# IMPORTANT: Set up the sectors first
analyzer.set_sectors()  # This was missing in the previous code

# Now get the full analysis
market_df, sector_df = analyzer.create_analysis_report()

# Then filter the sectors in various ways:

# Example 1: Get top 3 sectors with positive momentum and increasing volume
strong_sectors = analyzer.filter_sectors(
    sector_df,
    min_momentum=0,
    volume_trend='Increasing',
    top_n=3,
    sort_by='Overall Score'
)

# Example 2: Get all sectors outperforming SPY with minimum 2% return
outperformers = analyzer.filter_sectors(
    sector_df,
    outperforming_spy=True,
    min_return=2.0,
    sort_by='Return (%)'
)

# Example 3: Get sectors with strong relative strength and positive momentum
strong_rs = analyzer.filter_sectors(
    sector_df,
    min_rs=1.0,
    min_momentum=0,
    sort_by='Relative Strength'
)

# Example 4: Custom combination of filters
custom_filter = analyzer.filter_sectors(
    sector_df,
    min_return=0,
    min_rs=0,
    volume_trend='Increasing',
    sort_by='Overall Score',
    top_n=5
)

ib.disconnect()

print("=== Market Analysis ===")
display(market_df)

print("\n=== Sector Analysis ===")
display(sector_df)


try:
    # Print results
    print("\n=== Strong Sectors ===")
    display(strong_sectors)

    print("\n=== Outperformers ===")
    display(outperformers)

    print("\n=== Strong Relative Strength ===")
    display(strong_rs)

    print("\n=== Custom Filter ===")
    display(custom_filter)

except Exception as e:
    print(f"An error occurred: {str(e)}")

# 3. Scanner

In [ ]:
import scanner.scanner as scanner

stockbot = scanner.StockbotScanner(ib)
stockbot.multiscan(scan_code='TOP_PERC_GAIN', price=(1, 100), volume=100_000, change_perc=4, market_cap=100)
stockbot.scan_results_df

# 4. Fundamentals

In [ ]:
import fundamentals

from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

# # Connect to IB
# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)

try:
    stock_info = fundamentals.get_stock_info(ib, 'AAPL', current_volume=1000000)
    print(f"Current Price: ${stock_info.current_price:.2f}")
    print(f"52-week Range: ${stock_info.low_52week:.2f} - ${stock_info.high_52week:.2f}")
    print(f"Volume vs 10-day avg: {stock_info.volume_vs_10day_avg_pct:.2f}%")
except Exception as e:
    print(f"Error getting stock info: {e}")
finally:
    ib.disconnect()

stock_info.__dict__


In [ ]:
import my_ib_utils
import fundamentals

# Create rate limiter
rate_limiter = my_ib_utils.IBRateLimiter(ib, requests_per_second=2)

# Create Fundamentals instance
fund = fundamentals.Fundamentals(ib, rate_limiter)

# Get fundamentals for multiple stocks
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
data = fund.get_fundamentals(symbols)

# Convert to DataFrame and filter
df = fund.to_dataframe()
filtered_df = fund.filter_stocks({
    'pe_ratio': (0, 30),
    'market_cap': (1e11, None)
})

filtered_df

# 5. Daily Stock Data

In [1]:
import os

# Go up one level from scanner folder, then into data folder
current_dir = os.getcwd()  # Gets current directory (scanner)
parent_dir = os.path.dirname(current_dir)  # Goes up one level to stockbot4
data_folder = os.path.join(parent_dir, 'data')  # Goes into data folder
os.chdir(data_folder)

import historical_data as hd

symbol = 'ROKU'
interval = '1 day'
# data = hd.get_hist_data(symbol, '52 weeksAgo', 'now', interval)
data = hd.get_hist_data(symbol, '2022-12-30', '2024-11-14', interval)
display(data)

Loading data from historical_data/ROKU_1_day.csv
Stored data: 478 rows of data
Loading data from historical_data/ROKU_1_day.csv


,open,high,low,close,volume
date,,,,,
2022-12-30,40.30,40.97,39.05,40.75,42515
2023-01-03,41.45,42.37,39.00,40.51,51584
2023-01-04,40.93,43.28,39.84,42.47,68893
2023-01-05,42.59,44.25,40.93,42.81,78793
2023-01-06,42.75,44.46,41.30,43.60,56965
...,...,...,...,...,...
2024-11-08,71.06,74.14,70.76,73.98,30291
2024-11-11,74.61,77.21,73.85,77.00,25805
2024-11-12,77.12,77.74,75.55,76.30,20699


# 6. Daily TA

In [2]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

# Now you can import your modules using absolute imports
from frame.frame import Frame
from strategies import ta

In [9]:


f = Frame(symbol='TSLA')

# TA may add many several columns of data to the dataframe 
f.add_ta(ta.MA('close', 200), {'dash': 'solid', 'color': 'cyan', 'width': 2})
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'purple', 'width': 2})
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=2)
f.add_ta(ta.ACC('close', 5, 10, 10), {'dash': 'solid', 'color': 'grey', 'width': 1}, row=3)
f.add_ta(ta.VolDev('volume', 10), {'dash': 'solid', 'color': 'pink', 'width': 1}, row=3)
f.add_ta(ta.TrendDuration('MA_cl_50'), {'dash': 'solid', 'color': 'green', 'width': 1}, row=4)

f.load_ohlcv(data.iloc[100:])
f.update_ta_data()

ta_list = [
    ta.Breaks('close', 'MA_cl_200', True), # BreaksMA = 200
    ta.Breaks('close', 'MA_cl_50', True), # BreaksMA = 50
    ta.AboveBelow('close', 'MA_cl_50', True), # > MA50
    ta.AboveBelow('close', 'MA_cl_200', True), # > MA200
    ta.AboveBelow(3, 'TDUR_MA_cl_50', True), # MA50isRisingNthDays
    ta.AboveBelow(80, 'VDEV_10', True) # > 80% above MA10
]

for t in ta_list:
    f.add_ta(t)

f.load_ohlcv(f.data)
f.update_ta_data()


# Process all filters at once
result = ta.process_ta_filters(f.data, ta_list, 5)   
result

# f.setup_chart() 
# f.plot(trading_hours=True)

             open   high    low  close  volume  MA_cl_200  MA_cl_50  MA_vo_10  \
date                                                                            
2023-05-25  58.75  60.00  54.29  54.29   47222        NaN       NaN       NaN   
2023-05-26  54.90  56.45  53.93  56.10   31445        NaN       NaN       NaN   
2023-05-30  56.78  59.17  56.42  57.97   33382        NaN       NaN       NaN   
2023-05-31  57.51  58.31  56.30  58.20   33019        NaN       NaN       NaN   
2023-06-01  58.13  59.27  56.48  58.88   30579        NaN       NaN       NaN   
...           ...    ...    ...    ...     ...        ...       ...       ...   
2024-11-08  71.06  74.14  70.76  73.98   30291   65.92035   73.2768   48936.0   
2024-11-11  74.61  77.21  73.85  77.00   25805   65.86085   73.4626   50131.6   
2024-11-12  77.12  77.74  75.55  76.30   20699   65.77455   73.6870   50755.3   
2024-11-13  76.25  78.57  75.10  75.31   20926   65.70155   73.9232   47334.0   
2024-11-14  75.30  76.35  74

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2024-02-07,92.89,96.06,91.87,94.75,39558,NaN,94.0274,34138.4,7.47,15.875378,-7,True,True,False,True,True,5,False
2024-05-17,61.50,61.86,60.60,61.45,14528,76.73030,61.3974,27163.7,9.47,-46.516859,-77,True,True,False,True,True,5,False
2024-06-27,57.16,59.51,56.77,59.20,33846,73.33430,57.9254,26429.2,10.81,28.062900,1,True,True,False,True,True,5,False
2024-07-26,58.28,59.28,57.42,59.28,21820,72.27505,58.8060,28224.2,-22.13,-22.690457,-3,True,True,False,True,True,5,False
2024-07-31,57.30,59.11,56.94,58.70,20492,72.08275,58.5948,25091.4,-13.28,-18.330583,-6,True,True,False,True,True,5,False
2024-08-15,59.45,61.00,55.88,59.30,31276,71.69800,58.0256,31683.0,15.23,-1.284601,1,True,True,False,True,True,5,False
2024-11-08,71.06,74.14,70.76,73.98,30291,65.92035,73.2768,48936.0,3.54,-38.100785,4,True,True,True,False,True,5,False


In [4]:
f.load_ohlcv(data.iloc[100:])
f.update_ta_data()
f.setup_chart() 
f.data

             open   high    low  close  volume
date                                          
2023-05-25  58.75  60.00  54.29  54.29   47222
2023-05-26  54.90  56.45  53.93  56.10   31445
2023-05-30  56.78  59.17  56.42  57.97   33382
2023-05-31  57.51  58.31  56.30  58.20   33019
2023-06-01  58.13  59.27  56.48  58.88   30579
...           ...    ...    ...    ...     ...
2024-11-08  71.06  74.14  70.76  73.98   30291
2024-11-11  74.61  77.21  73.85  77.00   25805
2024-11-12  77.12  77.74  75.55  76.30   20699
2024-11-13  76.25  78.57  75.10  75.31   20926
2024-11-14  75.30  76.35  74.20  74.26   17070

[372 rows x 5 columns]


,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2023-05-25,58.75,60.00,54.29,54.29,47222,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-05-26,54.90,56.45,53.93,56.10,31445,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-05-30,56.78,59.17,56.42,57.97,33382,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-05-31,57.51,58.31,56.30,58.20,33019,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-06-01,58.13,59.27,56.48,58.88,30579,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-08,71.06,74.14,70.76,73.98,30291,65.92035,73.2768,48936.0,3.54,-38.100785,4,True,True,True,False,True,NaN,NaN
2024-11-11,74.61,77.21,73.85,77.00,25805,65.86085,73.4626,50131.6,22.44,-48.525481,5,False,True,True,False,True,NaN,NaN
2024-11-12,77.12,77.74,75.55,76.30,20699,65.77455,73.6870,50755.3,36.49,-59.218052,6,False,True,True,False,True,NaN,NaN


In [5]:
# import pandas as pd
# # max rows to display
# pd.set_option('display.max_rows', None)
# f.data  

In [6]:
f.plot(trading_hours=False)

# 7. Filter TA

In [7]:
# BreaksMA = 200 // yesterday below,  today above
# BreaksMA = 50 // yesterday below,  today above
# > MA200
# BreaksVolumeMA = 10 // breaks 10 day vol ma
# MA50isRisingNthDays = (3, 10) // viable range midpoint is best score
# With Segment trend
# With Mkt trend 


ta_1 = ta.Breaks('close', 'MA_cl_200', True) # BreaksMA = 200
ta_2 = ta.Breaks('close', 'MA_cl_50', True) # BreaksMA = 50
ta_3 = ta.AboveBelow('close', 'MA_cl_50', True) # > MA50
ta_4 = ta.AboveBelow('close', 'MA_cl_200', True) # > MA200
ta_5 = ta.AboveBelow(3, 'TDUR_MA_cl_50', True) # MA50isRisingNthDays
ta_6 = ta.AboveBelow(80, 'VDEV_10', True) # > 80% above MA10

f.add_ta(ta_1)
f.add_ta(ta_2)
f.add_ta(ta_3)
f.add_ta(ta_4)
f.add_ta(ta_5)
f.add_ta(ta_6)

ta_filter_names = [ta_1.name, ta_2.name, ta_3.name, ta_4.name, ta_5.name, ta_6.name]

df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

f.load_ohlcv(data)
f.update_ta_data()
f.data

             open   high    low  close  volume
date                                          
2022-12-30  40.30  40.97  39.05  40.75   42515
2023-01-03  41.45  42.37  39.00  40.51   51584
2023-01-04  40.93  43.28  39.84  42.47   68893
2023-01-05  42.59  44.25  40.93  42.81   78793
2023-01-06  42.75  44.46  41.30  43.60   56965
...           ...    ...    ...    ...     ...
2024-11-08  71.06  74.14  70.76  73.98   30291
2024-11-11  74.61  77.21  73.85  77.00   25805
2024-11-12  77.12  77.74  75.55  76.30   20699
2024-11-13  76.25  78.57  75.10  75.31   20926
2024-11-14  75.30  76.35  74.20  74.26   17070

[472 rows x 5 columns]


,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2022-12-30,40.30,40.97,39.05,40.75,42515,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-03,41.45,42.37,39.00,40.51,51584,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-04,40.93,43.28,39.84,42.47,68893,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-05,42.59,44.25,40.93,42.81,78793,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-06,42.75,44.46,41.30,43.60,56965,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-08,71.06,74.14,70.76,73.98,30291,65.92035,73.2768,48936.0,3.54,-38.100785,4,True,True,True,False,True,NaN,NaN
2024-11-11,74.61,77.21,73.85,77.00,25805,65.86085,73.4626,50131.6,22.44,-48.525481,5,False,True,True,False,True,NaN,NaN
2024-11-12,77.12,77.74,75.55,76.30,20699,65.77455,73.6870,50755.3,36.49,-59.218052,6,False,True,True,False,True,NaN,NaN


# 8. Daily Scores

In [8]:
df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2023-06-01,58.13,59.27,56.48,58.88,30579,NaN,58.8208,42705.7,7.60,-28.395975,-25,True,True,False,True,True,5,NaN
2023-11-02,70.27,79.12,69.00,77.57,332103,66.79200,71.4652,77477.6,24.07,328.643892,-21,True,True,True,True,False,5,NaN
2024-01-08,88.90,94.90,88.85,93.56,55274,74.14190,90.6266,42749.5,-18.88,29.297419,39,True,True,True,False,True,5,NaN
2024-02-07,92.89,96.06,91.87,94.75,39558,76.91170,94.0274,34138.4,7.47,15.875378,-7,True,True,True,True,True,6,NaN
2024-05-17,61.50,61.86,60.60,61.45,14528,76.73030,61.3974,27163.7,9.47,-46.516859,-77,True,True,False,True,True,5,NaN
2024-06-27,57.16,59.51,56.77,59.20,33846,73.33430,57.9254,26429.2,10.81,28.062900,1,True,True,False,True,True,5,NaN
2024-07-26,58.28,59.28,57.42,59.28,21820,72.27505,58.8060,28224.2,-22.13,-22.690457,-3,True,True,False,True,True,5,NaN
2024-07-31,57.30,59.11,56.94,58.70,20492,72.08275,58.5948,25091.4,-13.28,-18.330583,-6,True,True,False,True,True,5,NaN
2024-08-15,59.45,61.00,55.88,59.30,31276,71.69800,58.0256,31683.0,15.23,-1.284601,1,True,True,False,True,True,5,NaN


# 9. Final Watch List